## 1. Initialize settings

In [79]:
# change these to try this notebook out
BUCKET = 'cloudonair-ml-demo'
PROJECT = 'cloudonair-ml-demo'
REGION = 'us-central1'

In [80]:
import os

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [81]:
%%bash

gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## 2. Explore Natality dataset

In [82]:
# Call BigQuery and examine in dataframe
import pandas
from pandas.io import gbq

# Create SQL query using natality data after the year 2000
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 100
"""

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'], verbose=False)
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.053894,False,15,1,38
1,7.685314,False,16,1,41
2,5.249206,False,17,1,36
3,8.562754,False,18,1,39
4,6.062712,False,19,1,39


## 3. Training on Cloud ML Engine

In [83]:
%%bash

if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  # copy canonical set of preprocessed files if you didn't do previous notebook
  gsutil -m cp -R gs://cloud-training-demos/babyweight gs://${BUCKET}
fi

In [ ]:
%%bash

OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBNAME=babyweight_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME

gsutil -m rm -rf $OUTDIR

gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.4 \
  -- \
  --bucket=${BUCKET} \
  --output_dir=${OUTDIR} \
  --train_examples=200000

## 3. Get a saved model directory

In [84]:
import datetime
import os
import pytz
import re
import urllib

from google.cloud import storage

def get_saved_model_dir(bucket, pattern='.*/babyweight/.*/saved_model.pb'):
  created_time = datetime.datetime(datetime.MINYEAR, 1, 1, tzinfo=pytz.timezone('UTC'))
  recent_model_path = None

  p = re.compile(pattern)
  
  for blob in storage.Client().bucket(bucket).list_blobs():
    if p.match(urllib.unquote(blob.path)):
      if created_time < blob.time_created:
        recent_model_path = blob.path
        created_time = blob.time_created

  saved_model = urllib.unquote(recent_model_path.split('/')[-1])
  saved_model_dir = '/'.join(saved_model.split('/')[:-1])
  return 'gs://{0}/{1}'.format(os.environ['BUCKET'], saved_model_dir)

saved_model_dir = get_saved_model_dir(os.environ['BUCKET'])

if saved_model_dir:
  os.environ['SAVED_MODEL_DIR'] = saved_model_dir
  print(saved_model_dir)
else:
  print('wait until saved_model.pb is written by Cloud ML Engine...')

gs://cloudonair-ml-testing/babyweight/trained_model/export/exporter/1526957260


## 4. Testing an evaluation pipeline with DirectRunner

### Configure a DirectRunner job

In [85]:
import os
import datetime

os.environ['RUNNER'] = 'DirectRunner'
os.environ['JOB_NAME'] = 'evaluate-ml-model-{0}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
os.environ['BQ_DATASET'] = 'model_evaluation'
os.environ['BQ_TABLE'] = 'sample'
os.environ['OUTPUT_TABLE'] = '{}:{}.{}'.format(os.environ['PROJECT'], os.environ['BQ_DATASET'], os.environ['BQ_TABLE'])

In [86]:
%%bash

# Create BQ dataset if it doesn't exist.
if ! bq ls ${BQ_DATASET} > /dev/null; then
  bq mk ${BQ_DATASET}
fi

### Execute a DirectRunner job

In [87]:
%%bash

cd $(pwd)/evaluator

python -m main \
  --datasize=100 \
  --year_from=1974 \
  --year_to=1978 \
  --saved_model_dir=${SAVED_MODEL_DIR} \
  --output_table=${OUTPUT_TABLE} \
  --project=${PROJECT} \
  --runner=DirectRunner \
  --region=${REGION} \
  --job_name=${JOB_NAME}

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:32: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/__init__.py:95: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *
INFO:root:Running pipeline with DirectRunner.
/usr/local/envs/py2env/lib/python2.7/site-packages/oauth2client/contrib/gce.py:99: UserWarning: You have requested explicit scopes to be used wi

### Show results of a DirectRunner Job

In [88]:
import pandas
from pandas.io import gbq

query = """
SELECT * FROM `{0}` LIMIT 10
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'], verbose=False)
df.head()

,is_male,mother_age,plurality,gestation_weeks,weight_true,weight_predicted,weight_residual,model,testdata,time_inference
0,True,20,Single(1),40,9.499719,7.500028,1.999691,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,0.930071
1,True,21,Single(1),40,7.561856,7.637622,-0.075766,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,0.897169
2,True,27,Single(1),99,5.732019,14.236391,-8.504372,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,0.864029
3,True,18,Single(1),40,7.438397,7.021181,0.417216,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,0.951052
4,True,25,Single(1),39,7.749249,7.118065,0.631183,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,0.956059


In [89]:
import pandas
from pandas.io import gbq

query = """
SELECT
  model,
  testdata,
  SQRT(SUM(POW(weight_residual,2))/COUNT(*)) AS RMSE,
  AVG(time_inference) AS AVG_MSEC
FROM
  `{0}`
GROUP BY
  model, testdata
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'], verbose=False)
df.head()

,model,testdata,RMSE,AVG_MSEC
0,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,4.043845,3.156292
1,gs://cloudonair-ml-testing/babyweight/trained_...,1974-1978,3.794716,3.072488


## 5. Running an evaluation pipeline with DataflowRunner

### Configure a Dataflow job

In [90]:
import os
import datetime

os.environ['RUNNER'] = 'DataflowRunner'
os.environ['STAGING_LOCATION'] = 'gs://{0}/babyweight/staging'.format(os.environ['BUCKET'])
os.environ['TEMP_LOCATION'] = 'gs://{0}/babyweight/temp'.format(os.environ['BUCKET'])
os.environ['JOB_NAME'] = 'evaluate-ml-model-{0}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
os.environ['BQ_DATASET'] = 'model_evaluation'
os.environ['BQ_TABLE'] = 'prediction_results'
os.environ['OUTPUT_TABLE'] = '{}:{}.{}'.format(os.environ['PROJECT'], os.environ['BQ_DATASET'], os.environ['BQ_TABLE'])

In [91]:
%%bash

# Create BQ dataset if it doesn't exist.
if ! bq ls ${BQ_DATASET} > /dev/null; then
  bq mk ${BQ_DATASET}
fi

### Execute a Dataflow job

In [ ]:
%%bash

cd $(pwd)/evaluator

python -m main \
  --datasize=1000000 \
  --year_from=1984 \
  --year_to=1988 \
  --saved_model_dir=${SAVED_MODEL_DIR} \
  --output_table=${OUTPUT_TABLE} \
  --project=${PROJECT} \
  --runner=${RUNNER} \
  --region=${REGION} \
  --staging_location=${STAGING_LOCATION} \
  --temp_location=${TEMP_LOCATION} \
  --job_name=${JOB_NAME} \
  --setup_file=$(pwd)/setup.py

In [ ]:
import pandas
from pandas.io import gbq

query = """
SELECT * FROM `{0}` LIMIT 10
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'], verbose=False)
df.head()

In [ ]:
query = """
SELECT
  model,
  testdata,
  SQRT(SUM(POW(weight_residual,2))/COUNT(*)) AS RMSE,
  AVG(time_inference) AS AVG_MSEC
FROM
  `{0}`
GROUP BY
  model, testdata
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'], verbose=False)
df

## 6. Visualize Optimizing and Satisficing Metric with DataStudio

Now, you can create sharable dashboard using Google Data Studio like below. In the similar way, you can also create a dashboard for error analysis so that both business owners and engineers do simple error analysis.

<img src="dashboard1.png">